https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [1]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from flights_delay.feature_processing import FlightDelayFeatureProcessing
from helpers.outliers import MyOutliers
from helpers.feature_engineering import makeBinaryClassification, getUniqueValuesPerFeature
from sklearn.model_selection import train_test_split
from helpers.plot_helper import plotDistsPerClass, plotBothDists
from flights_delay.feature_prev_delays import FeaturePrevDelays
from flights_delay.feature_airline_delays import FeatureAirlineDelays
from helpers.my_one_hot_encoder import MyOneHotEncoder
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from datetime import datetime
from sklearn.dummy import DummyClassifier
from helpers.feature_engineering import dateStrToDayYear, getMappingOfSimilarCategoricalColumns, \
    compareSimilarCategoricalColumns 
from helpers.my_one_hot_encoder import MyOneHotEncoder
from helpers.py_helpers import is_number
from scipy.stats import skew, kurtosis
from helpers.outliers import MyOutliers
from sklearn.preprocessing import StandardScaler
from helpers.plot_helper import scatter_2d_label
from sklearn.decomposition import PCA # Import the PCA module
from sklearn.manifold import TSNE
from sklearn.decomposition import KernelPCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from helpers.performance_issues import subsample_keeping_class_proportions
from sklearn.metrics import f1_score, precision_score

In [ ]:
plt.style.use('ggplot')

In [14]:
random_state = 0 # Ensure reproducible results

In [4]:
data_relative_path = '../Data'

In [9]:
target_col = 'IS_DELAYED'

# Flying to New York City - Classifying

In [5]:
path_data = os.path.realpath(os.path.join(os.getcwd(), data_relative_path, 'train_data_v2_engineered.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/train_data_v2_engineered.csv'

In [6]:
df = pd.read_csv(path_data, delimiter = ',', header=0)
df.shape

(31186, 43)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31186 entries, 0 to 31185
Data columns (total 43 columns):
QUARTER               31186 non-null int64
MONTH                 31186 non-null int64
DAY_OF_MONTH          31186 non-null int64
DAY_OF_WEEK           31186 non-null int64
DEP_TIME              31186 non-null float64
DEP_DELAY             31186 non-null float64
DEP_DELAY_NEW         31186 non-null float64
DEP_DEL15             31186 non-null float64
DEP_DELAY_GROUP       31186 non-null float64
CRS_ELAPSED_TIME      31186 non-null float64
PREV_DELAYS_30        31186 non-null float64
PREV_DELAYS_60        31186 non-null float64
PREV_DELAYS_120       31186 non-null float64
PREV_DELAYS_180       31186 non-null float64
PREV_DELAYS_360       31186 non-null float64
PREV_DELAYS_720       31186 non-null float64
AIRLINE_DELAYS_30     31186 non-null float64
AIRLINE_DELAYS_60     31186 non-null float64
AIRLINE_DELAYS_120    31186 non-null float64
AIRLINE_DELAYS_180    31186 non-null float64

In [11]:
df.sample(10, random_state=random_state)

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,CRS_ELAPSED_TIME,...,ORIGIN_3,ORIGIN_4,ORIGIN_5,DEST_0,DEST_1,DEST_2,DEST_3,DEST_4,DEST_5,IS_DELAYED
16192,1,1,0,0,268.878788,-1.0,0.0,0.0,-1.0,328.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,False
10496,0,0,0,0,383.878788,-1.0,0.0,0.0,-1.0,315.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,False
18939,1,1,1,1,193.878788,-1.0,0.0,0.0,-1.0,346.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,True
30189,0,0,0,0,1803.878788,-5.0,0.0,0.0,-1.0,147.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
25267,0,1,1,0,1350.878788,11.0,11.0,0.0,0.0,408.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
20921,1,1,0,0,594.878788,-5.0,0.0,0.0,-1.0,356.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,False
21224,1,1,0,0,885.878788,-5.0,0.0,0.0,-1.0,169.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
7213,0,0,1,0,1152.878788,-6.0,0.0,0.0,-1.0,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
7726,0,0,0,0,770.878788,-2.0,0.0,0.0,-1.0,177.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,False
22552,0,0,0,0,686.878788,7.0,7.0,0.0,0.0,389.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False


In [10]:
XX = df.drop(labels=[target_col], axis=1)
yy = df[target_col]
XX.shape, yy.shape

((31186, 42), (31186,))

In [13]:
columns_one_hot_encoded = np.array([col for col in df.columns if is_number(col[-1])])
columns_not_one_hot = np.array([col for col in df.columns if not is_number(col[-1])])

NameError: name 'is_number' is not defined